# Pixel induced current
In this notebook we load a track dataset and we calculate the current induced on the pixels placed at the anode.

In [245]:
# This is need so you can import larndsim without doing python setup.py install
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [246]:
from larndsim import detsim

import importlib
importlib.reload(detsim)
from larndsim import consts, quenching, drifting
importlib.reload(drifting)
importlib.reload(quenching)

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d

import pickle
import numpy as np
import numba as nb
from tqdm import tqdm_notebook as progress_bar

Here we load the dataset with the tracks and we store the column indeces corresponding to the each variable.

In [247]:
tracks = pickle.load(open('FakeTrackDataSet.p', 'rb'))
tracks['t'] = 0.
tracks['t_start'] = 0.
tracks['t_end'] = 0.

tracks['dx'] = np.sqrt(pow(tracks['x_end']-tracks['x_start'], 2) +
                    pow(tracks['y_end']-tracks['y_start'], 2) +
                    pow(tracks['z_end']-tracks['z_start'], 2))
tracks['x'] = (tracks['x_end']+tracks['x_start'])/2
tracks['y'] = (tracks['y_end']+tracks['y_start'])/2
tracks['z'] = (tracks['z_end']+tracks['z_start'])/2
tracks['dE'] = np.abs(tracks['dE'])*1e3
tracks['dEdx'] = tracks['dE']/tracks['dx']
tracks['NElectrons'] = 0.
tracks['longDiff'] = 0.
tracks['tranDiff'] = 0.

Here we pass our columnar data through the quenching and drifting stages, so we calculate the amount of electrons that reach the anode.

In [248]:
tracks_np = tracks.to_numpy()
cols = nb.typed.Dict()
for i, c, in enumerate(tracks.columns):
    cols[c] = i

In [249]:
quenching.Quench(tracks_np, cols, mode="box")

In [250]:
drifting.Drift(tracks_np, cols)

In [251]:
indeces = {c:i for i, c, in enumerate(tracks.columns)}
selectedTracks = tracks_np[(tracks_np[:,cols['trackID']] > 6) & (tracks_np[:,cols['trackID']] < 9)]

Here we instantiate a TPC class with a determined number of pixels per axis and a determined sampling in time.

In [252]:
tpc = detsim.TPC(t_sampling=1, n_pixels=1000, **indeces)

Here we calculate the current induced by each track on the pixels.

In [253]:
for t in progress_bar(selectedTracks, desc='Processing tracks...'):
    img = tpc.calculateCurrent(t)

In [256]:
%load_ext line_profiler

In [259]:
%lprun -f tpc.calculateCurrent tpc.calculateCurrent(selectedTracks[0])

Timer unit: 1e-06 s

Total time: 1.25219 s
File: /Users/soleti/dune/larnd-sim/larndsim/detsim.py
Function: calculateCurrent at line 207

Line #      Hits         Time  Per Hit   % Time  Line Contents
   207                                               def calculateCurrent(self, track):
   208         1       7400.0   7400.0      0.6          pixelsIDs = self.getPixels(track)
   209                                           
   210         1          5.0      5.0      0.0          xs, xe = track[self.ixStart], track[self.ixEnd]
   211         1          1.0      1.0      0.0          ys, ye = track[self.iyStart], track[self.iyEnd]
   212         1          3.0      3.0      0.0          zs, ze = track[self.izStart], track[self.izEnd]
   213                                           
   214         1          7.0      7.0      0.0          startVector = np.array([xs, ys, zs])
   215         1          3.0      3.0      0.0          endVector = np.array([xe, ye, ze])
   216         1    

## Track and pixel display
Here we show the selected tracks and the pixels affected by them. The color of the pixel correspond to the integrated induced current.

In [254]:
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

currents = np.array([sum(tpc.getPixelResponse(p)) for p in tpc.activePixels])
norm = mpl.colors.LogNorm(vmin=min(currents[currents>0]), vmax=max(currents))
cmap = cm.viridis
m = cm.ScalarMappable(norm=norm, cmap=cmap)


for i,p in enumerate(list(tpc.activePixels.keys())):
    rect = plt.Rectangle((p[0]*tpc.x_pixel_size+consts.tpcBorders[0][0],
                          p[1]*tpc.y_pixel_size+consts.tpcBorders[1][0]),
                         tpc.x_pixel_size,tpc.y_pixel_size,
                         linewidth=0,
                         edgecolor='grey', 
                         facecolor=m.to_rgba(currents[i]), label='Pixel')
    ax.add_patch(rect)
    art3d.pathpatch_2d_to_3d(rect, z=150, zdir="z")
    
for t in selectedTracks:
    ax.plot((t[indeces['x_start']], t[indeces['x_end']]), 
            (t[indeces['y_start']], t[indeces['y_end']]),
            (t[indeces['z_start']], t[indeces['z_end']]),
            c='r')
    ax.scatter((t[indeces['x_start']], t[indeces['x_end']]), 
               (t[indeces['y_start']], t[indeces['y_end']]),
               (t[indeces['z_start']], t[indeces['z_end']]),
               c='r')

ax.set_xlabel("$x$ coordinate [cm]")
ax.set_ylabel("$y$ coordinate [cm]")
ax.set_zlabel("$z$ coordinate [cm]")
ax.set_zlim3d(150,50)
ax.set_xlim3d(-80,-130)
ax.set_ylim3d(55,100)

_ = plt.colorbar(m,fraction=0.046, pad=0.04, label='Current integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [255]:
%matplotlib widget
fig,ax = plt.subplots(1,1, constrained_layout=True)

for i,p in enumerate(list(tpc.activePixels.keys())):
    rect = plt.Rectangle((p[0]*tpc.x_pixel_size+consts.tpcBorders[0][0],
                          p[1]*tpc.y_pixel_size+consts.tpcBorders[1][0]),
                         tpc.x_pixel_size,tpc.y_pixel_size,
                         linewidth=0.3,
                         edgecolor='grey', 
                         facecolor=m.to_rgba(currents[i]), label='Pixel')
    ax.add_patch(rect)
    
for t in selectedTracks:
    ax.plot((t[indeces['x_start']], t[indeces['x_end']]), 
            (t[indeces['y_start']], t[indeces['y_end']]),
            c='r',
            zorder=10)
#     ax.scatter((t[indeces['x_start']], t[indeces['x_end']]), 
#                (t[indeces['y_start']], t[indeces['y_end']]),
#                c='r',
#                zorder=10)

ax.set_xlabel("$x$ coordinate [cm]")
ax.set_ylabel("$y$ coordinate [cm]")
ax.set_aspect("equal")
_ = plt.colorbar(m,fraction=0.046, pad=0.04, label='Current integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Pixel induced current
We are able to plot the current induced on each pixel. The `getPixelFromCoordinates` allows to get the pixel ID corresponding to a specific $(x,y)$ pair of coordinates.

In [12]:
fig, ax = plt.subplots(1,1, constrained_layout=True)
pixelID = tpc.getPixelFromCoordinates(-111.8,72.3)
current = tpc.getPixelResponse(pixelID)
ax.plot(tpc.anode_t, current, label='Current')
ax.set_xlim(1100,1500) 
ax.set_title("Pixel (%i,%i) at (%.2f, %2.f)" % (pixelID[0], 
                                      pixelID[1], 
                                      (pixelID[0] + 0.5)*tpc.x_pixel_size-150, 
                                      (pixelID[1] + 0.5)*tpc.y_pixel_size-150))
ax.set_xlabel("Time [$\mathrm{\mu}$s]")
_ = ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …